Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d (characters)' % len(text))

Data size 100000000 (characters)


In [4]:
print('First 800 characters:')
print(text[:800])

First 800 characters:
 anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization of society it has also been taken up as a positive label by self defined anarchists the word anarchism is derived from the greek without archons ruler chief king anarchism as a political philosophy is the belief that rulers are unnecessary and should be abolished although there are differing interpretations of what this means anarchism also refers to related social movements that advocate the elimination of authoritarian institutions particularly the state the word anarchy as most anarchis


Create a small validation set.

In [5]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)

print("#characters for training: {}".format(train_size))
print('First 110 characters for training:\n', train_text[:110])

print("\n#characters for validation: {}".format(valid_size))
print('First 110 characters for validation:\n', valid_text[:110])

#characters for training: 99999000
First 110 characters for training:
 ons anarchists advocate social relations based upon voluntary association of autonomous individuals mutual aid

#characters for validation: 1000
First 110 characters for validation:
  anarchism originated as a term of abuse first used against early working class radicals including the diggers


Utility functions to map characters to vocabulary IDs and back.

In [72]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' ' = 27
first_letter = ord(string.ascii_lowercase[0])  # 97

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print('vocabulary size: {}\n'.format(vocabulary_size))
print('id  ', char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print('char', id2char(1), id2char(26), id2char(0), '', 'ï')

vocabulary size: 27

Unexpected character: ï
id   1 26 0 0
char a z    ï


Function to generate a training batch for the LSTM model.

In [7]:
batch_size = 64
num_unrollings = 10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print('Training batch:')
print(batches2string(train_batches.next()))
print('\nTraining batch:')
print(batches2string(train_batches.next()))

print('\nValidation batch:')
print(batches2string(valid_batches.next()))
print('\nValidation batch:')
print(batches2string(valid_batches.next()))

Training batch:
['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']

Training batch:
['ists

In [8]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [9]:
num_nodes = 64

def weight_variable(shape):
  return tf.Variable(tf.truncated_normal(shape, -0.1, 0.1))

def bias_variable(shape):
  return tf.Variable(tf.zeros(shape))

def saved_variable(shape):
  return tf.Variable(tf.zeros(shape), trainable=False)
  
  
graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = weight_variable([vocabulary_size, num_nodes])
  im = weight_variable([num_nodes, num_nodes])
  ib = bias_variable([1, num_nodes])
  
   # Forget gate: input, previous output, and bias.
  fx = weight_variable([vocabulary_size, num_nodes])
  fm = weight_variable([num_nodes, num_nodes])
  fb = bias_variable([1, num_nodes])
  
   # Memory cell: input, previous output, and bias.
  cx = weight_variable([vocabulary_size, num_nodes])
  cm = weight_variable([num_nodes, num_nodes])
  cb = bias_variable([1, num_nodes])

   # Output gate: input, previous output, and bias.
  ox = weight_variable([vocabulary_size, num_nodes])
  om = weight_variable([num_nodes, num_nodes])
  ob = bias_variable([1, num_nodes])
  
  
  # Variables saving state across unrollings.
  saved_output = saved_variable([batch_size, num_nodes])
  saved_state = saved_variable([batch_size, num_nodes])
  
  # Classifier weights and biases.
  w = weight_variable([num_nodes, vocabulary_size])
  b = bias_variable([vocabulary_size])
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings] # a list of 10 tensors
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for itrain in train_inputs:
    output, state = lstm_cell(itrain, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(0, train_labels), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [10]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print('Step %d' % step)
      print('Average loss: %.3f, learning rate: %.2f' % (mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Step 0
Average loss: 3.295, learning rate: 10.00
Minibatch perplexity: 26.97
mewalxxt  inznuyr am gha  lgrlhrien ieb o dug ej udmbzeroe vhcsuen cnanlhptnijsz
sx  mailuunpyjc ejm sekjbu huntg jmiy yfoe dohck gcmp rfrgi omc r nkebrtfetoganw
jcrcybc eiife tvoovelfak mvdfaayrek kebtmwil hci bindwhemdd  czamsbe gxmeynuxbov
pmtyy tvpe rkfl weg gl fnew   dym ekwfnvwtgniozoep  cocfieowqe raffc cqzax ldcxo
x tedi rqw  ekdx   yaviutoqeprar did qi fkayklerfsnmbuuecco enmhjon u sg  sfrhqy
Validation set perplexity: 20.19
Step 100
Average loss: 2.605, learning rate: 10.00
Minibatch perplexity: 11.26
Validation set perplexity: 10.54
Step 200
Average loss: 2.254, learning rate: 10.00
Minibatch perplexity: 8.55
Validation set perplexity: 8.63
Step 300
Average loss: 2.108, learning rate: 10.00
Minibatch perplexity: 7.55
Validation set perplexity: 8.01
Step 400
Average loss: 2.007, learning rate: 10.00
Minibatch perplexity: 7.32
Validation set perplexity: 7.76
Step 500
Average loss: 1.942, l

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [51]:
def lstm_cell(i, o, state):
  """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
  Note that in this formulation, we omit the various connections between the
  previous state and the gates."""
  input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
  forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
  update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
  state = forget_gate * state + input_gate * tf.tanh(update)
  output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)

  return output_gate * tf.tanh(state), state



# ix:(27,64)  im:(64,64)  ib:(1,64)
# fx:(27,64)  fm:(64,64)  fb:(1,64)
# cx:(27,64)  cm:(64,64)  cb:(1,64)
# ox:(27,64)  om:(64,64)  ob:(1,64)
# i :(64,27)  o :(64,64)  state:(64,64)

# input_gate:(64,64)
# forget_gate:(64,64)
# update:(64,64)
# state:(64,64)
# output_gate:(64,64)
# output: (64,64)


# saved_output = saved_variable([batch_size, num_nodes])
# saved_state = saved_variable([batch_size, num_nodes])
outputs = []
# output = saved_output (64,64)
# state = saved_state
for itrain in train_inputs:  #(64,27)
  output, state = lstm_cell(itrain, output, state)
  outputs.append(output)

i = itrain
o = output
input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
state = forget_gate * state + input_gate * tf.tanh(update)
output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)

In [64]:
first_letter

97

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---